In [1]:
from sklearn.datasets import load_wine

In [2]:
wine = load_wine()

In [ ]:
print(wine.DESCR)

In [15]:
x = wine.data
y = wine.target

x.shape, y.shape

((178, 13), (178,))

In [16]:
import pandas as pd

In [ ]:
pd.DataFrame(y, columns=wine.target_names).head()

In [20]:
wine.target_names

array(['class_0', 'class_1', 'class_2'], dtype='<U7')

## 문제

1. 전처리 : 필수) training set / test set 구분
2. 모델링 : Functional API
    - 1) 히든 레이어 없이 성능 보기
    - 2) 히든 레이어 추가하고 성능 보기 (자율)
3. 학습 : epochs=100, validation set 10%, 5번까지만 참을래!

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.1, random_state=2022)

In [23]:
train_x.shape, train_y.shape

((160, 13), (160,))

In [ ]:
# x 전처리 : min-max scaling

# max_n , min_n = train_x.max(), train_x.min()
# max_n , min_n

In [26]:
# train_x_mm = (train_x - min_n) / (max_n - min_n)

In [ ]:
# train_x_mm.max(), train_x_mm.min()

In [ ]:
# pd.DataFrame(train_x_mm, columns=wine.feature_names).describe()

In [29]:
from sklearn.preprocessing import MinMaxScaler

In [30]:
mm_scaler = MinMaxScaler()

In [32]:
train_x_mm = mm_scaler.fit_transform(train_x)
test_x_mm = mm_scaler.transform(test_x)

In [33]:
pd.DataFrame(train_x_mm, columns=wine.feature_names).describe()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
count,160.000000,160.000000,160.000000,160.000000,160.000000,160.000000,160.000000,160.000000,160.000000,160.000000,160.000000,160.000000,160.000000
mean,0.521875,0.313303,0.534191,0.495496,0.357485,0.448772,0.354101,0.445637,0.365024,0.317625,0.393831,0.508585,0.329462
std,0.218885,0.221658,0.150895,0.182454,0.165266,0.216551,0.208664,0.231837,0.175543,0.198660,0.183752,0.263348,0.226615
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.349462,0.171937,0.453209,0.368715,0.222222,0.253448,0.190401,0.264151,0.260252,0.159343,0.260163,0.257547,0.157454
50%,0.528226,0.219368,0.529412,0.494413,0.333333,0.467241,0.371308,0.396226,0.345426,0.283276,0.406504,0.575472,0.272468
75%,0.699597,0.446640,0.632353,0.608939,0.456790,0.627586,0.515295,0.589623,0.471609,0.415316,0.520325,0.717925,0.466833
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [36]:
# y 전처리 : One-hot Encoding
import numpy as np

In [39]:
class_n = len(np.unique(train_y))

In [40]:
from tensorflow.keras.utils import to_categorical

In [41]:
train_y = to_categorical(train_y, class_n)
test_y = to_categorical(test_y, class_n)

In [42]:
train_x.shape, train_y.shape

((160, 13), (160, 3))

In [46]:
## Functional API
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.backend import clear_session
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

In [60]:
# 클리어 세션
clear_session()

# 사슬처럼 엮기
il = Input(shape=(13,))
hl = Dense(64, activation='relu')(il)
hl = Dense(64, activation='relu')(hl)
ol = Dense(3, activation='softmax')(hl)

# 모델의 시작과 끝 지정
model = Model(il, ol)

# 컴파일
model.compile(loss=keras.losses.categorical_crossentropy, metrics=['accuracy'],
              optimizer='adam')

# 요약
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 13)]              0         
                                                                 
 dense (Dense)               (None, 64)                896       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 3)                 195       
                                                                 
Total params: 5,251
Trainable params: 5,251
Non-trainable params: 0
_________________________________________________________________


In [61]:
# 학습
from tensorflow.keras.callbacks import EarlyStopping

In [62]:
es = EarlyStopping(monitor='val_loss',
                   min_delta=0,
                   verbose=1,
                   patience=5,
                   restore_best_weights=True)

In [59]:
## 히든 레이어 없이 학습

model.fit(train_x_mm, train_y, validation_split=0.1, callbacks=[es],
          epochs=100, verbose=1)

Epoch 1/100
5/5 [==============================] - 1s 43ms/step - loss: 1.1092 - accuracy: 0.3194 - val_loss: 0.9524 - val_accuracy: 0.5000
Epoch 2/100
5/5 [==============================] - 0s 8ms/step - loss: 1.0982 - accuracy: 0.3194 - val_loss: 0.9498 - val_accuracy: 0.5000
Epoch 3/100
5/5 [==============================] - 0s 13ms/step - loss: 1.0874 - accuracy: 0.3264 - val_loss: 0.9473 - val_accuracy: 0.5000
Epoch 4/100
5/5 [==============================] - 0s 8ms/step - loss: 1.0775 - accuracy: 0.3264 - val_loss: 0.9450 - val_accuracy: 0.5000
Epoch 5/100
5/5 [==============================] - 0s 7ms/step - loss: 1.0684 - accuracy: 0.3264 - val_loss: 0.9429 - val_accuracy: 0.5000
Epoch 6/100
5/5 [==============================] - 0s 8ms/step - loss: 1.0598 - accuracy: 0.3264 - val_loss: 0.9411 - val_accuracy: 0.5000
Epoch 7/100
5/5 [==============================] - 0s 11ms/step - loss: 1.0519 - accuracy: 0.3333 - val_loss: 0.9396 - val_accuracy: 0.5000
Epoch 8/100
5/5 [=======

In [63]:
## 히든 레이어 추가한 학습

model.fit(train_x_mm, train_y, validation_split=0.1, callbacks=[es],
          epochs=100, verbose=1)

Epoch 1/100
5/5 [==============================] - 1s 48ms/step - loss: 1.1061 - accuracy: 0.4236 - val_loss: 1.0893 - val_accuracy: 0.3125
Epoch 2/100
5/5 [==============================] - 0s 10ms/step - loss: 1.0398 - accuracy: 0.4514 - val_loss: 1.0047 - val_accuracy: 0.4375
Epoch 3/100
5/5 [==============================] - 0s 10ms/step - loss: 0.9805 - accuracy: 0.6667 - val_loss: 0.9304 - val_accuracy: 0.8125
Epoch 4/100
5/5 [==============================] - 0s 10ms/step - loss: 0.9259 - accuracy: 0.8611 - val_loss: 0.8689 - val_accuracy: 0.8750
Epoch 5/100
5/5 [==============================] - 0s 10ms/step - loss: 0.8709 - accuracy: 0.9236 - val_loss: 0.8084 - val_accuracy: 1.0000
Epoch 6/100
5/5 [==============================] - 0s 8ms/step - loss: 0.8142 - accuracy: 0.9514 - val_loss: 0.7509 - val_accuracy: 0.9375
Epoch 7/100
5/5 [==============================] - 0s 8ms/step - loss: 0.7575 - accuracy: 0.9375 - val_loss: 0.6945 - val_accuracy: 0.9375
Epoch 8/100
5/5 [=====